In [1]:
import pytimber
import yaml
import re
import tfs
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime, timedelta
from scipy.signal import find_peaks

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import lossmaps as lm

In [4]:
def parse_beam_plane(option):
    
    # Map to determine beam
    beam_map = {'B1': 'b1', 'B2': 'b2'}
    # Map to determine plane
    plane_map = {'H': 'horizontal', 'V': 'vertical'}
    
    # Extract beam and plane parts
    beam_key = option[:2]  # First two characters
    plane_key = option[2]  # Last character
    
    # Look up beam and plane
    beam = beam_map.get(beam_key.upper())
    plane = plane_map.get(plane_key.upper())
    
    if beam and plane:
        return beam, plane
    else:
        raise ValueError("Invalid option. Valid options are: B1H, B1V, B2H, B2V.")

In [5]:
parse_beam_plane('B1V')

('b1', 'vertical')

In [6]:
def get_utc_time(start_time, end_time):
    
    # Localize the timestamps to Swiss time (Europe/Zurich)
    tStart = start_time.tz_localize('Europe/Zurich')  # Localize to Swiss time (CET/CEST)
    tEnd = end_time.tz_localize('Europe/Zurich')

    # Convert to UTC
    tStart_utc = tStart.astimezone('UTC')
    tEnd_utc = tEnd.astimezone('UTC')
    
    return tStart_utc, tEnd_utc

In [7]:
ldb = pytimber.LoggingDB(spark_session=spark)
yaml_path = '/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/colldbs/injection.yaml'

In [8]:
def find_column_with_max_difference(df):
    # Exclude the 'time' column from consideration
    columns_to_check = df.columns.drop('time')
    
    # Compute the absolute difference between the first and last value for each column
    differences = df[columns_to_check].iloc[-1] - df[columns_to_check].iloc[0]
    absolute_differences = differences.abs()
    
    # Find the column with the maximum difference
    max_diff_column = absolute_differences.idxmax()
    
    return max_diff_column

In [9]:
# Load collimators 
def load_data(start_time, end_time, plane, beam):
    """
    Load and process collimator data.

    Parameters:
        start_time: Start timestamp.
        end_time: End timestamp.
        plane: 'horizontal' or 'vertical'.
        beam: 'b1' or 'b2'.
    
    Returns:
        ref_col: Reference collimator.
        ref_df: DataFrame with time and the reference collimator gap.
    """
    # Load the file  
    with open(yaml_path, 'r') as file:
        f = yaml.safe_load(file)

    # Create a data frame for beam 1
    cols = pd.DataFrame(f['collimators'][beam]).loc[['angle']].T
    cols = cols.reset_index().rename(columns={'index': 'name'})
    
    if plane == 'horizontal': angle = 0
    elif plane == 'vertical': angle = 90
    cols = cols[cols['angle'] == angle].dropna()

    # Get a list of collimator names to load from timber
    names = cols['name'].str.upper().to_list()

    for i, name in enumerate(names): names[i]=name+':MEAS_LVDT_GD'

    cols = ldb.get(names, start_time, end_time)
    
    data = {'time': cols[next(iter(cols))][0]}
    df_test = pd.DataFrame(data)
    df_test['time'] = df_test['time'].astype(int)

    for key, (timestamps, values) in cols.items():
        # Compute the difference between consecutive values
        differences = np.diff(values)

        # Check if there are any non-zero differences
        if np.any(differences > 0.1):

            df = pd.DataFrame({
                'time': cols[key][0],
                key: cols[key][1]
            })
            df['time'] = df['time'].astype(int)

            merged = pd.merge(df_test, df, on='time')
            df_test = merged.copy()

    #df_test['time'] = df_test['time'].astype(int)
    
    ref_col = find_column_with_max_difference(df_test)
    ref_df = df_test[['time', ref_col]]
    ref_df.rename(columns={ref_col: 'gap'}, inplace=True)
    
    return ref_col, ref_df, df_test

In [10]:
ref_col, ref_df, df_test = load_data(start_time, end_time, 'horizontal', 'b1')

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/cvmfs/sft.cern.ch/lcg/views/LCG_105a_nxcals_pro/x86_64-el9-gcc13-opt/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/cvmfs/sft.cern.ch/lcg/views/LCG_105a_nxcals_pro/x86_64-el9-gcc13-opt/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.12-9a1bc/x86_64-el9-gcc13-opt/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [11]:
def process_dataframe(df):
    # Round all columns except 'time' to one decimal place
    columns_to_round = [col for col in df.columns if col != 'time']
    df[columns_to_round] = df[columns_to_round].round(1)
    
    # Determine the column with the most unique values
    # Exclude 'time' column from consideration
    unique_counts = {col: df[col].nunique() for col in columns_to_round}
    column_with_most_steps = max(unique_counts, key=unique_counts.get)
    
    return df[['time', column_with_most_steps]], column_with_most_steps

In [12]:
def get_blm_df(start_time, end_time):

    # Now pass the localized and converted timestamps to the function
    data = lm.blm.get_BLM_data(start_time, end_time, spark=spark)
    names = lm.blm.get_BLM_names(start_time, spark=spark)

    vals_df = pd.DataFrame(data['vals'].tolist(), columns=names)

    # Concatenate 'timestamp' with the new DataFrame
    data_expanded = pd.concat([data['timestamp'], vals_df], axis=1)
    
    data_expanded['time'] = pd.to_datetime(data_expanded['timestamp']).astype(int) / 10**9

    # Drop the old timestamp column if needed
    data_expanded = data_expanded.drop(columns=['timestamp'])

    # Reorder the columns if desired
    data_expanded = data_expanded[['time'] + [col for col in data_expanded.columns if col != 'time']]
    
    # Sort the DataFrame by 'column_name' in increasing order
    data_expanded = data_expanded.sort_values(by='time', ascending=True)

    # Reset index if needed to maintain a clean, sequential index
    data_expanded = data_expanded.reset_index(drop=True)
    
    mqx_columns = data_expanded.filter(like='MQX')
    max_col_name = mqx_columns.loc[:, mqx_columns.columns != 'timestamp'].idxmax(axis=1)
    bottleneck = max_col_name.to_list()[-1]

    return data_expanded, bottleneck

In [60]:
data_expanded, bottleneck = get_blm_df(start_time, end_time)

/eos/user/m/morwat/.local/lib/python3.9/site-packages/lossmaps/nxcals.py:83: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.



In [25]:
# Now pass the localized and converted timestamps to the function
data = lm.blm.get_BLM_data(start_time, end_time, spark=spark)
names = lm.blm.get_BLM_names(start_time, spark=spark)

vals_df = pd.DataFrame(data['vals'].tolist(), columns=names)

# Concatenate 'timestamp' with the new DataFrame
data_expanded = pd.concat([data['timestamp'], vals_df], axis=1)

data_expanded['time'] = pd.to_datetime(data_expanded['timestamp']).astype(int) / 10**9

# Drop the old timestamp column if needed
data_expanded = data_expanded.drop(columns=['timestamp'])

# Reorder the columns if desired
data_expanded = data_expanded[['time'] + [col for col in data_expanded.columns if col != 'time']]

# Sort the DataFrame by 'column_name' in increasing order
data_expanded = data_expanded.sort_values(by='time', ascending=True)

# Reset index if needed to maintain a clean, sequential index
data_expanded = data_expanded.reset_index(drop=True)

mqx_columns = data_expanded.filter(like='MQX')
max_col_name = mqx_columns.loc[:, mqx_columns.columns != 'timestamp'].idxmax(axis=1)
bottleneck = max_col_name.to_list()[-1]

/eos/user/m/morwat/.local/lib/python3.9/site-packages/lossmaps/nxcals.py:83: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  pd.Timestamp(time, unit=_NXCALS_timestamp_unit, tz='CET').round(freq='S') for time in df['timestamp']


In [49]:
data_expanded

,time,BLMEL.01R1.B2I10_BPMSW.1R1,BLMQI.01R1.B2I30_MQXA,BLMQI.01R1.B1E10_MQXA,BLMQI.01R1.B2I20_MQXA,BLMQI.01R1.B1E20_MQXA,BLMQI.02R1.B2I30_MQXB,BLMQI.01R1.B1E30_MQXA,BLMQI.02R1.B2I23_MQXB,BLMQI.02R1.B1E21_MQXB,...,BLMQI.02L1.B1I22_MQXB,BLMQI.02L1.B2E21_MQXB,BLMQI.02L1.B1I23_MQXB,BLMQI.01L1.B2E30_MQXA,BLMQI.02L1.B1I30_MQXB,BLMQI.01L1.B2E20_MQXA,BLMQI.01L1.B1I20_MQXA,BLMQI.01L1.B2E10_MQXA,BLMEL.01L1.B1I10_BPMSW.1L1,BLMQI.01L1.B1I30_MQXA
0,1.656430e+09,1.160431e-06,0.000001,1.013596e-06,1.041214e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,9.859772e-07,0.000001,0.000009,0.000001
1,1.656430e+09,1.199112e-06,0.000001,1.043976e-06,1.043976e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.002548e-06,0.000001,0.000008,0.000001
2,1.656430e+09,5.028534e-07,0.000001,1.016357e-06,1.035690e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.043976e-06,0.000001,0.000009,0.000001
3,1.656430e+09,1.199112e-06,0.000001,9.832153e-07,9.887390e-07,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.030167e-06,0.000001,0.000009,0.000001
4,1.656430e+09,1.044388e-06,0.000001,9.942627e-07,9.942627e-07,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,9.859772e-07,0.000001,0.000009,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,1.656431e+09,1.199112e-06,0.000001,1.052261e-06,1.038452e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.019119e-06,0.000001,0.000009,0.000001
1167,1.656431e+09,1.121750e-06,0.000001,1.027405e-06,1.030167e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,9.915009e-07,0.000001,0.000009,0.000001
1168,1.656431e+09,1.315155e-06,0.000001,1.038452e-06,1.016357e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,9.887390e-07,0.000001,0.000009,0.000001
1169,1.656431e+09,1.508560e-06,0.000001,1.032928e-06,1.021881e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.027405e-06,0.000001,0.000008,0.000001


In [80]:
# Remove columns that are constant within 1e-5 precision
filtered_df = data_expanded.drop('time', axis=1)
filtered_df = filtered_df.loc[:, ~filtered_df.columns.str.contains("TC")]
filtered_df = filtered_df.loc[:, filtered_df.std() > 1e-6]

# Determine the second half of the data
halfway_index = len(filtered_df) // 2
second_half = filtered_df.iloc[halfway_index:]

# Calculate the sum of each column in the second half and find the column with the maximum sum
bottleneck1 = second_half.sum().idxmax()

In [83]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# Create a subplot with a secondary y-axis
fig = go.Figure()

fig.add_trace(go.Scatter(x=data_expanded.time, y=data_expanded['BLMAI.04R8.B2E10_MBXB']/data_expanded['BLMAI.04R8.B2E10_MBXB'].max(), mode='lines', name=1))

fig.add_trace(go.Scatter(x=data_expanded.time, y=data_expanded['BLMEL.06R7.B2I10_TCHSS.6R7.B2']/data_expanded['BLMEL.06R7.B2I10_TCHSS.6R7.B2'].max(), mode='lines', name=0))

fig.update_layout(
    width=800,  # Set width in pixels
    height=600  # Set height in pixels
)

# Show the figure
fig.show()

In [30]:
data_expanded

,time,BLMEL.01R1.B2I10_BPMSW.1R1,BLMQI.01R1.B2I30_MQXA,BLMQI.01R1.B1E10_MQXA,BLMQI.01R1.B2I20_MQXA,BLMQI.01R1.B1E20_MQXA,BLMQI.02R1.B2I30_MQXB,BLMQI.01R1.B1E30_MQXA,BLMQI.02R1.B2I23_MQXB,BLMQI.02R1.B1E21_MQXB,...,BLMQI.02L1.B1I22_MQXB,BLMQI.02L1.B2E21_MQXB,BLMQI.02L1.B1I23_MQXB,BLMQI.01L1.B2E30_MQXA,BLMQI.02L1.B1I30_MQXB,BLMQI.01L1.B2E20_MQXA,BLMQI.01L1.B1I20_MQXA,BLMQI.01L1.B2E10_MQXA,BLMEL.01L1.B1I10_BPMSW.1L1,BLMQI.01L1.B1I30_MQXA
0,1.656430e+09,1.160431e-06,0.000001,1.013596e-06,1.041214e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,9.859772e-07,0.000001,0.000009,0.000001
1,1.656430e+09,1.199112e-06,0.000001,1.043976e-06,1.043976e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.002548e-06,0.000001,0.000008,0.000001
2,1.656430e+09,5.028534e-07,0.000001,1.016357e-06,1.035690e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.043976e-06,0.000001,0.000009,0.000001
3,1.656430e+09,1.199112e-06,0.000001,9.832153e-07,9.887390e-07,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.030167e-06,0.000001,0.000009,0.000001
4,1.656430e+09,1.044388e-06,0.000001,9.942627e-07,9.942627e-07,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,9.859772e-07,0.000001,0.000009,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,1.656431e+09,1.199112e-06,0.000001,1.052261e-06,1.038452e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.019119e-06,0.000001,0.000009,0.000001
1167,1.656431e+09,1.121750e-06,0.000001,1.027405e-06,1.030167e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,9.915009e-07,0.000001,0.000009,0.000001
1168,1.656431e+09,1.315155e-06,0.000001,1.038452e-06,1.016357e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,9.887390e-07,0.000001,0.000009,0.000001
1169,1.656431e+09,1.508560e-06,0.000001,1.032928e-06,1.021881e-06,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,1.027405e-06,0.000001,0.000008,0.000001


In [13]:
def load_blm(ref_col, start_time, end_time):

    # Split the string by '.' and ':'
    split_string = ref_col.replace(':', '.').split('.')
    
    try:
        blm_string = 'BLMTI.0'+re.search(r'([0-9]+[RL][0-9]+)', split_string[1]).group()+'.'+split_string[2]+'E10_'+ref_col.split(':')[0]+':LOSS_RS09'

        blm = ldb.get(blm_string, start_time, end_time)

        data = {
                    'time': blm[blm_string][0],
                    'loss': blm[blm_string][1]
                }
    except:
        blm_string = 'BLMTI.0'+re.search(r'([0-9]+[RL][0-9]+)', split_string[1]).group()+'.'+split_string[2]+'I10_'+ref_col.split(':')[0]+':LOSS_RS09'

        blm = ldb.get(blm_string, start_time, end_time)

        data = {
                    'time': blm[blm_string][0],
                    'loss': blm[blm_string][1]
                }

    df = pd.DataFrame(data)
    df['time'] = df['time'].astype(int)
        
    return blm_string, df

In [14]:
def load_data_given_ref_col(start_time, end_time, ref_col):
    
    col = ldb.get(ref_col+':MEAS_LVDT_GD', start_time, end_time)
    
    df = pd.DataFrame({
                    'time': col[ref_col+':MEAS_LVDT_GD'][0],
                    'gap': col[ref_col+':MEAS_LVDT_GD'][1]
                })

    df['time'] = df['time'].astype(int)
    
    return ref_col+':MEAS_LVDT_GD', df

In [15]:
def fun(start_time, end_time, plane, beam, ref_col=None):
    
    if ref_col is not None:
        ref_col, df_col = load_data_given_ref_col(start_time, end_time, ref_col)
    else:   
        ref_col, df_col = load_data(start_time, end_time, plane, beam)
        
    ref_blm, df_blm = load_blm(ref_col, start_time, end_time)
    df = pd.concat([df_blm, df_col.drop(df_col.columns[0], axis=1)], axis=1)
    print(ref_col, ref_blm)
    
    # Calculate the difference in 'gap'
    gap_diff = df['gap'].diff().abs()  # Get the absolute difference

    # Create a mask for where the difference is greater than 0.1
    split_mask = gap_diff > 0.1
    
    dfs = []

    # Track start index for each split segment
    segment_start_indices = []
    start_index = 0
    for i in range(len(split_mask)):
        if split_mask[i]:
            # If there's a split point, slice the DataFrame
            segment = df.iloc[start_index:i]
            if segment.shape[0] > 5:  # Ensure segment has more than 5 rows
                dfs.append(segment)
                segment_start_indices.append(start_index)  # Record the start index of the segment
            start_index = i  # Update the start index for the next segment

    # Append the last segment after the loop
    dfs.append(df.iloc[start_index:])
    segment_start_indices.append(start_index)

    # List to store peak indices relative to df1
    peak_indices = []
    
    # Process each segment and get the index of the highest peak in each
    for segment, segment_start in zip(dfs, segment_start_indices):
        # Use find_peaks to locate peaks
        peaks, _ = find_peaks(segment.loss)

        if peaks.size > 0:  # Ensure there are peaks to evaluate
            # Get the index of the highest peak within the segment
            highest_peak_idx = peaks[np.argmax(segment.loss.iloc[peaks])]

            # Convert to index in df1 and store
            peak_indices.append(segment_start + highest_peak_idx)
        else:
            # Handle case where no peaks are found, take the highest value
            highest_peak_index = np.argmax(segment.loss)

            # Convert to index in df1 and store
            peak_indices.append(segment_start + highest_peak_index)
    
    peak_indices = [i for i in peak_indices if i != 0]
            
    corresponding_times = df['time'].iloc[peak_indices].values
    
    peaks = pd.DataFrame({
            'time': corresponding_times,
            'peaks': peak_indices
            })
        
    return ref_col, ref_blm, df_col, df_blm, peaks

In [90]:
def get_bunches(start_time, end_time, ref_col):

    beam = check_which_beam(ref_col)
    
    if beam == 'B1': name = 'LHC.BCTFR.A6R4.B1:BUNCH_INTENSITY'
    elif beam == 'B2': name = 'LHC.BCTFR.A6R4.B2:BUNCH_INTENSITY'
    
    bunches = ldb.get(name, start_time, end_time)

    # Extract time and bunch intensity arrays from the dictionary
    time_array = bunches[name][0]
    bunch_intensity_array = np.stack(bunches[name][1], axis=1)

    # Initialize the DataFrame with the time array
    df = pd.DataFrame({'time': time_array})

    # Create a DataFrame for the bunch intensity data, transposed so each bunch is a column
    bunch_data_df = pd.DataFrame(bunch_intensity_array.T, columns=[f'Bunch {n}' for n in range(bunch_intensity_array.shape[0])])

    # Concatenate the time DataFrame and the bunch data DataFrame along columns
    df = pd.concat([df, bunch_data_df], axis=1)

    # Remove entries with only zeros
    df = df.drop(columns=[col for col in df if (df[col] == 0).all()])
    
    # Calculate the difference between the first and last values for each column, ignoring "Time"
    # Keep only the numeric columns (ignores "Time")
    drops = df.iloc[0] - df.iloc[-1]

    # Find the column with the maximum decrease
    column_with_max_drop = drops.idxmax()
    
    return df, column_with_max_drop

In [66]:
df, column_with_max_drop=get_bunches(start_time, end_time, ref_col)

In [69]:
smoothed_bunch_intensity

array([1.0378149e+10, 1.0378149e+10, 1.0378149e+10, ..., 9.9497964e+09,
       9.9497964e+09, 9.9497964e+09], dtype=float32)

In [16]:
def find_bottleneck(blm_data):
    """
    Find the column with the maximum sum in the second half of the data.
    
    Parameters:
        blm_data (DataFrame): Input DataFrame with beam loss data.
        
    Returns:
        str: Name of the column with the maximum sum in the second half.
    """
    # Exclude columns with "TC" in their names
    filtered_df = blm_data.loc[:, blm_data.columns.str.contains("MQX")]

    # Determine the second half of the data
    halfway_index = len(filtered_df) // 2
    second_half = filtered_df.iloc[halfway_index:]

    # Calculate the sum of each column in the second half and find the column with the maximum sum
    max_sum_column = second_half.sum().idxmax()

    return max_sum_column

In [17]:
def check_which_beam(ref_col):
    
    beam = re.search('B[12]', ref_col).group()
    
    return beam

In [18]:
def find_all_peaks(peaks, df_blm_col, df_blm_mqx, prominence, min_separation):
    
    all_peaks1, _ = find_peaks(df_blm_col.loss/df_blm_col.loss.max(), prominence=prominence, distance=min_separation)
    all_peaks2, _ = find_peaks(df_blm_mqx[bottleneck]/df_blm_mqx[bottleneck].max(), prominence=prominence, distance=min_separation)
    all_peaks = np.concatenate([all_peaks1, all_peaks2, np.array(peaks.peaks.values)])
    
    # Sort
    all_peaks = np.sort(all_peaks)

    # Remove peaks that are too close
    all_peaks = all_peaks[np.insert(np.diff(all_peaks) >= min_separation, 0, True)]
    
    if len(peaks.peaks.values)>len(all_peaks):
        all_peaks = peaks.peaks.values
        
    corresponding_times = df_blm_mqx['time'].iloc[all_peaks].values
    
    data = pd.DataFrame({
            'time': corresponding_times,
            'peaks': all_peaks
            })
    
    return data

In [19]:
def find_all_peaks_test(peaks, df_blm_col, df_blm_mqx, prominence, min_separation):

    # Find peaks in the normalized columns
    all_peaks1, _ = find_peaks(df_blm_col.loss / df_blm_col.loss.max(), prominence=prominence, distance=min_separation)
    all_peaks2, _ = find_peaks(df_blm_mqx[bottleneck] / df_blm_mqx[bottleneck].max(), prominence=prominence, distance=min_separation)

    # Combine `all_peaks1` and `all_peaks2`
    candidate_peaks = np.concatenate([all_peaks1, all_peaks2])
    
    # Start with all `peaks`, ensuring they're always included
    final_peaks = list(peaks.peaks.values)
    
    # Add candidate peaks if they are far enough from existing peaks
    for candidate in np.sort(candidate_peaks):
        if all(abs(candidate - existing_peak) >= min_separation for existing_peak in final_peaks):
            final_peaks.append(candidate)
    
    # Sort final peaks
    final_peaks = np.sort(final_peaks)
    
    # Get corresponding times for final peaks
    corresponding_times = df_blm_mqx['time'].iloc[final_peaks].values

    # Create a DataFrame with the results
    data = pd.DataFrame({
        'time': corresponding_times,
        'peaks': final_peaks
    })

    return data

In [20]:
def smooth_bunch_intensity(all_peaks, df_bunches, column_with_max_drop):   
    
    bunch_intensity = df_bunches[column_with_max_drop].values
    time = df_bunches['time'].values

    # Create segments based on filtered transition indices
    segments = []
    start_idx = 0

    for idx in all_peaks.peaks.values:
        if idx > start_idx:  # Ensure segment has at least one point
            segments.append((start_idx, idx))
        start_idx = idx

    # Append the final segment if it has data
    if start_idx < len(bunch_intensity):
        segments.append((start_idx, len(bunch_intensity)))

    # Smooth each step segment by averaging within the segment
    smoothed_bunch_intensity = np.copy(bunch_intensity)

    for start, end in segments:
        if end > start:  # Ensure segment is non-empty
            smoothed_bunch_intensity[start:end] = np.mean(bunch_intensity[start:end])

    # Handle any remaining NaNs if they appear
    smoothed_bunch_intensity = np.nan_to_num(smoothed_bunch_intensity, nan=np.nanmean(bunch_intensity))
    
    return smoothed_bunch_intensity

In [21]:
def get_protons_lost(peaks, all_peaks, smoothed_bunch_intensity):
    
    unique_values = np.unique(smoothed_bunch_intensity)
    unique_values = unique_values[::-1]  # Sort unique values in decreasing order

    # Step 2: Use indices to retrieve corresponding times from blm_data.time
    corresponding_times = all_peaks.time.values
    
    protons_lost = []
    
    for i in range(len(unique_values)-1):
        protons_lost.append(unique_values[i]-unique_values[i+1])
    
    # Step 3: Create a DataFrame with unique values and corresponding times
    df = pd.DataFrame({
        'protons_lost': protons_lost,
        'time': corresponding_times
    })
    
    # Step 4: Filter `df` to only keep rows with times in `df_blm.time.iloc[peaks]`
    target_times = df_blm_col['time'].iloc[peaks.peaks.values].values #this can be simpler
    df_filtered = df[df['time'].isin(target_times)]

    return pd.merge(peaks, df, on='time', how='left')

In [22]:
def load_optics_and_rescale(df_col, path, plane, ref_col, peaks):
    
    df_col.reset_index(drop=True, inplace=True)
    gaps = df_col.iloc[peaks.peaks.values].gap
    
    df = tfs.read(path)
    
    if plane == 'horizontal': column = 'BETX'
    if plane == 'vertical': column = 'BETY'   
        
    beta = df[df.NAME == ref_col.split(':')[0]][column].values[0]
    
    gamma = tfs.reader.read_headers(path)['GAMMA']
    emittance = 3.5e-6
    
    sigma = np.sqrt(beta * emittance / gamma)
    
    return gaps*1e-3/sigma

In [23]:
def normalise_blm_data(df_blm, column, peaks, all_peaks, protons_lost):
    
    # Convert both Series to NumPy arrays to ensure alignment
    loss_values = df_blm[column].iloc[peaks.peaks.values].values
    protons_lost_values = protons_lost.protons_lost.values 

    # Perform the normalized division
    normalised_blm = loss_values / protons_lost_values
    #normalised_blm = normalised_blm/normalised_blm.max()
    
    return normalised_blm

In [84]:
# Start of the measurements
start_time = pd.to_datetime('2022-06-28 17:20:00')
# End of the measurements
end_time = pd.to_datetime('2022-06-28 17:39:30')

start_time, end_time = get_utc_time(start_time, end_time)

In [85]:
%%time
print('Loading collimators...')
ref_col, ref_blm, df_col, df_blm_col, peaks = fun(start_time, end_time, 'horizontal', 'b1', 'TCTPH.4L5.B1')

Loading collimators...
TCTPH.4L5.B1:MEAS_LVDT_GD BLMTI.04L5.B1I10_TCTPH.4L5.B1:LOSS_RS09
CPU times: user 301 ms, sys: 65.3 ms, total: 367 ms
Wall time: 50.3 s


In [86]:
%%time
print('Loading BLMs...')
df_blm_mqx, bottleneck = get_blm_df(start_time, end_time)

Loading BLMs...


/eos/user/m/morwat/.local/lib/python3.9/site-packages/lossmaps/nxcals.py:83: FutureWarning:

'S' is deprecated and will be removed in a future version, please use 's' instead.



CPU times: user 1.7 s, sys: 355 ms, total: 2.05 s
Wall time: 18.3 s


In [91]:
%%time
print('Loading bunches...')
df_bunches, column_with_max_drop = get_bunches(start_time, end_time, ref_col)

Loading bunches...
CPU times: user 725 ms, sys: 84.7 ms, total: 809 ms
Wall time: 9.86 s


In [92]:
prominence, min_separation = 0.1, 10
all_peaks = find_all_peaks_test(peaks, df_blm_col, df_blm_mqx, prominence, min_separation)

In [93]:
smoothed_bunch_intensity = smooth_bunch_intensity(all_peaks, df_bunches, column_with_max_drop)

In [94]:
protons_lost = get_protons_lost(peaks, all_peaks, smoothed_bunch_intensity)

path = '/eos/project-c/collimation-team/machine_configurations/LHC_run3/2023/MADX/levelling.20/all_optics_B1.tfs'
plane = 'horizontal'
gaps = load_optics_and_rescale(df_col, path, plane, ref_col, peaks)

normalised_col_blm = normalise_blm_data(df_blm_col, 'loss', peaks, all_peaks, protons_lost)
normalised_bottleneck_blm = normalise_blm_data(df_blm_mqx, bottleneck, peaks, all_peaks, protons_lost)

ValueError: All arrays must be of the same length

In [ ]:
def normalise_again(array):
    return array/array.max()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a subplot with a secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add the first trace (Gap) to the primary y-axis
fig.add_trace(go.Scatter(x=df_col.time, y=df_col.gap, mode='lines', name='Gap'), secondary_y=False)

# Add the second trace (Loss) to the secondary y-axis
fig.add_trace(go.Scatter(x=df_blm_col.time, y=df_blm_col.loss / df_blm_col.loss.max(), mode='lines', name='Loss'), secondary_y=True)
fig.add_trace(go.Scatter(x=df_blm_col.time.iloc[peaks.peaks.values], y=df_blm_col.loss.iloc[peaks.peaks.values] / df_blm_col.loss.max(), mode='markers', name='PEAKS'), secondary_y=True)

# Add the third trace (Bunch Intensity) intended for a tertiary y-axis
fig.add_trace(go.Scatter(x=df_bunches.time, y=df_bunches[column_with_max_drop], mode='lines', name='Bunch intensity', yaxis="y3"))

#for i in dfs:
 #   fig.add_trace(go.Scatter(x=i.time, y=i.loss))

# Add another trace for Loss at bottleneck (on secondary y-axis)
fig.add_trace(go.Scatter(x=df_blm_mqx.time, y=df_blm_mqx[bottleneck] / df_blm_mqx[bottleneck].max(), mode='lines', name='Loss at bottleneck'), secondary_y=True)

# Update layout for the tertiary y-axis with adjustments
fig.update_layout(
    title='Gap, Loss, and Bunch Intensity Over Time',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Gap'),  # Primary y-axis title
    yaxis2=dict(title='Loss', overlaying='y', side='right', tickfont=dict(size=10)),  # Secondary y-axis
    yaxis3=dict(  # Tertiary y-axis configuration
        title="Bunch Intensity",
        titlefont=dict(color="blue", size=10),
        tickfont=dict(color="blue", size=9),  # Smaller font for tick labels
        anchor="free",
        overlaying="y",
        side="right",
        position=0.85  
    )
)

# Show the figure
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# Create a subplot with a secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add the first trace (Gap) to the primary y-axis
fig.add_trace(go.Scatter(x=df_col.time, y=df_col.gap, mode='lines', name='Gap'), secondary_y=False)

# Add the second trace (Loss) to the secondary y-axis
fig.add_trace(go.Scatter(x=df_blm_col.time, y=df_blm_col.loss / df_blm_col.loss.max(), mode='lines', name='Loss'), secondary_y=True)
fig.add_trace(go.Scatter(x=df_blm_col.time.iloc[peaks.peaks.values], y=df_blm_col.loss.iloc[peaks.peaks.values] / df_blm_col.loss.max(), mode='markers', name='Peaks'), secondary_y=True)
#fig.add_trace(go.Scatter(x=df_blm_col.time.iloc[all_peaks.peaks.values], y=df_blm_col.loss.iloc[all_peaks.peaks.values] / df_blm_col.loss.max(), mode='markers', name='ALL Peaks'), secondary_y=True)

fig.add_trace(go.Scatter(x=df_bunches.time, y=smoothed_bunch_intensity, mode='lines', name='Smoothed Bunch Intensity', yaxis="y3"))

# Add another trace for Loss at bottleneck (on secondary y-axis)
fig.add_trace(go.Scatter(x=df_blm_mqx.time, y=df_blm_mqx[bottleneck] / df_blm_mqx[bottleneck].max(), mode='lines', name='Loss at bottleneck'), secondary_y=True)
fig.add_trace(go.Scatter(x=df_blm_mqx.time.iloc[all_peaks.peaks.values], y=df_blm_mqx[bottleneck].iloc[all_peaks.peaks.values] / df_blm_mqx[bottleneck].max(), mode='markers', name='All peaks'), secondary_y=True)

#fig.add_trace(go.Scatter(x=df_blm_mqx.time.iloc[all_peaks1], y=df_blm_mqx[bottleneck].iloc[all_peaks1] / df_blm_mqx[bottleneck].max(), mode='markers', name='1'), secondary_y=True)
#fig.add_trace(go.Scatter(x=df_blm_mqx.time.iloc[all_peaks2], y=df_blm_mqx[bottleneck].iloc[all_peaks2] / df_blm_mqx[bottleneck].max(), mode='markers', name='2'), secondary_y=True)

# Update layout for the tertiary y-axis with adjustments
fig.update_layout(
    title='Gap, Loss, and Bunch Intensity Over Time',
    xaxis=dict(title='Time'),
    yaxis=dict(title='Gap'),  # Primary y-axis title
    yaxis2=dict(title='Loss', overlaying='y', side='right', tickfont=dict(size=10)),  # Secondary y-axis
    yaxis3=dict(  # Tertiary y-axis configuration
        title="Bunch Intensity",
        titlefont=dict(color="blue", size=10),
        tickfont=dict(color="blue", size=9),  # Smaller font for tick labels
        anchor="free",
        overlaying="y",
        side="right",
        position=0.85  
    )
)

# Show the figure
fig.show()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# Create a subplot with a secondary y-axis
fig = go.Figure()

fig.add_trace(go.Scatter(x=gaps, y=normalise_again(normalised_col_blm[3:-2]), mode='lines+markers', name='coll'))

fig.add_trace(go.Scatter(x=gaps, y=normalise_again(normalised_bottleneck_blm[3:-2]), mode='lines+markers', name='bottleneck'))

fig.update_layout(
    width=800,  # Set width in pixels
    height=600  # Set height in pixels
)

# Show the figure
fig.show()